# EE499 Final Project
# Premier League Predictor
Hojeong Lee and Nate Chism

In [2]:
import numpy as np
import autograd.numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from autograd import grad 
from autograd import hessian
import math
import copy
import pandas as pd
import sys
import time
from IPython.display import clear_output
readDataPath = './readData/'
sys.path.append('./')

from sklearn.linear_model import Perceptron

In [3]:
season_data = pd.read_csv('./readData/df_full_premierleague.csv')
season_csv_to_array = np.array(season_data)
# print(range(0, len(house_csv_to_array) -1))
print(season_csv_to_array.shape)
print(season_csv_to_array)

(4070, 114)
[[0 'https://www.premierleague.com/match/7186' '10/11' ... 0.78 -4.0 55.6]
 [1 'https://www.premierleague.com/match/7404' '10/11' ... 0.32 17.0 60.2]
 [2 'https://www.premierleague.com/match/7255' '10/11' ... 0.38 9.0 66.7]
 ...
 [4067 'https://www.premierleague.com/match/59178' '20/21' ... 0.69 4.0
  64.1]
 [4068 'https://www.premierleague.com/match/59182' '20/21' ... 0.42 6.0
  54.2]
 [4069 'https://www.premierleague.com/match/59052' '20/21' ... 0.87 5.0
  71.1]]


In [4]:
# 2, 4, 5, 6, 8 - 31, 38 - 49, 56  
print(season_csv_to_array[5][100])

nan


In [5]:
i, j = season_csv_to_array.shape
print(i)
print(j)
match_target_scores = np.zeros((i,3))
# match_teams = np.array((i, 3))
for it_1 in range(0, i):
    # store game unique id
    match_target_scores[it_1][0] = season_csv_to_array[it_1][0]
    #store game unique id, home team, away team
    # match_teams[it_1][0] = season_csv_to_array[it_1][0]
    # match_teams[it_1][1] = season_csv_to_array[it_1][4]
    # match_teams[it_1][2] = season_csv_to_array[it_1][5]
    if season_csv_to_array[it_1][34] > 0:
        match_target_scores[it_1][1] = 1
    elif season_csv_to_array[it_1][34] < 0:
        match_target_scores[it_1][1] = -1
    else:
        match_target_scores[it_1][1] = 0

print(match_target_scores[678])

4070
114
[678.  -1.   0.]


In [6]:
season_data_cleaned = np.zeros((i, 102))

# print(season_csv_to_array[5])
season_csv_to_array[pd.isnull(season_csv_to_array)] = 0
# print(season_csv_to_array[5])
# 2, 4, 5, 6, 8 - 31, 38 - 49, 56  
for it in range(0, i):
    ct = 0
    for it2 in range(0, j):
        
        # if(np.isnan(season_csv_to_array[it][it2])):
        #     print(season_csv_to_array[it][it2])

        if it2 not in [1, 2, 3, 4, 5, 6, 7, 32, 33, 34, 35, 36, 37]:
            season_data_cleaned[it][ct] = season_csv_to_array[it][it2]
            # print(ct)
            ct = ct + 1

        elif it2 == 2:
            season_number = (int)(season_csv_to_array[it][it2][0] + season_csv_to_array[it][it2][1])
            season_data_cleaned[it][ct] = season_number
            ct = ct + 1

        
        

# print(season_data_cleaned)
print(season_data_cleaned[5])


[  5.   10.   15.    5.    9.    4.  609.   65.6   0.   19.    4.   23.
 820.    2.   32.    3.    7.    0.  313.   34.4   0.    9.    4.   21.
 527.    2.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0. ]


In [7]:
print("it: ", it, " ct: ", ct, " it2: ", it2)

it:  4069  ct:  102  it2:  113


In [8]:
print(season_data_cleaned[4000][1])

20.0


In [9]:
season_data_cleaned[123][0]

123.0

In [10]:
seasons_10to18 = np.zeros((3419, 102))
season_19 = np.zeros((380, 102))
season_20 = np.zeros((271, 102))

Y_train = np.zeros((3419,))
Y_test1 = np.zeros((380,))
Y_test2 = np.zeros((271,))

ct1 = 0
ct2 = 0
ct3 = 0
for it in range(0, i):
    target_index = season_data_cleaned[it][0]
    if season_data_cleaned[it][1] < 19:
        seasons_10to18[ct1] = season_data_cleaned[it]
        Y_train[ct1] = match_target_scores[it][1]
        ct1 = ct1 + 1
    elif season_data_cleaned[it][1] == 19:
        season_19[ct2] = season_data_cleaned[it]
        Y_test1[ct2] = match_target_scores[it][1]
        ct2 = ct2 + 1
        # print(it)
    elif season_data_cleaned[it][1] == 20:
        season_20[ct3] = season_data_cleaned[it]
        Y_test2[ct3] = match_target_scores[it][1]
        ct3 = ct3 + 1
        # print(it)
    else:
        print('THIS SHOULD NEVER HAPPEN')

In [11]:
print(seasons_10to18.shape)
print(season_19.shape)
print(season_20.shape)

(3419, 102)
(380, 102)
(271, 102)


In [12]:
X_train = seasons_10to18
X_test1 = season_19
X_test2 = season_20



In [13]:
print(season_csv_to_array[1][4])

Liverpool


In [14]:
#model 1: Perceptron

# perc_clf = Perceptron(tol=1e-3, random_state=13)
# perc_clf.fit(X_train, Y_train)
# Perceptron()



In [15]:
# perc_clf.score(X_train, Y_train)

In [16]:
from sklearn.linear_model import SGDClassifier
linear_clf = SGDClassifier()

In [17]:
linear_clf.fit(X_train, Y_train)

SGDClassifier()

In [18]:
linear_clf.score(X_train,Y_train)

0.6074875694647558

In [19]:
print ("\nThe coefficients of the linear boundary are:", linear_clf.coef_)
print ("\nThe point of intersection of the line are:",linear_clf.intercept_)


The coefficients of the linear boundary are: [[-1.62026494e+01 -1.06183102e+02 -2.82393437e+03 -1.90588704e+02
  -1.38066500e+02 -1.53747248e+02  2.39078038e+02  5.95462784e+02
   1.50639566e+02 -1.46995749e+03 -1.80717661e+03 -5.21284066e+02
  -2.61970508e+02  1.17071852e+02  2.37638066e+03  6.65186366e+01
  -3.50788545e+02  5.57722235e+01 -9.68126091e+01 -1.70094369e+03
  -1.09599180e+02  1.86683937e+03  2.40245199e+03  6.61912624e+02
   2.64247893e+02 -4.60221666e+00  4.14908810e+02 -6.43717263e+01
  -1.45128767e+02 -2.87401313e+01  1.91808054e+02 -9.75361535e+01
  -6.93416458e+00 -2.47869696e+02 -1.20039570e+02 -4.40531770e+01
  -1.65656077e+01 -1.52679724e+01  1.43185873e+01 -4.03192135e+00
  -1.75690807e+02  2.74391748e+01 -3.93062514e+00  1.07464131e+01
  -2.61382183e+02 -9.97893418e+02  1.85296155e+02 -3.00116716e+01
  -3.25950808e+00 -1.02335269e+02  1.01386358e+02  4.89637896e+00
   3.26090773e+02  1.49085725e+02  2.02300634e+02  4.69188871e+01
   9.72396189e+01 -2.63370151e

In [20]:
linear_clf.score(X_test1,Y_test1)

0.5473684210526316

In [21]:
linear_clf.score(X_test2,Y_test2)

0.4797047970479705

In [22]:
y_pred1 = linear_clf.predict(X_test1)
y_pred2 = linear_clf.predict(X_test2)

In [23]:
standing = {}
indexofteams = []
for i in range(len(season_csv_to_array)):
    season_number = (int)(season_csv_to_array[i][2][0] + season_csv_to_array[i][2][1])
    if season_number == 19:
        indexofteams.append(i)
        if (season_csv_to_array[i][4]) in standing:
            standing[season_csv_to_array[i][4]] = 0
        else:
            standing[season_csv_to_array[i][4]] = 0

In [24]:
i= 3419
for ind in range(len(y_pred1)):
    if y_pred1[ind] == 1:
        standing[season_csv_to_array[i][4]] += 3
    elif y_pred1[ind] == 0:
        standing[season_csv_to_array[i][4]] += 1
        standing[season_csv_to_array[i][5]] += 1
    else:
        standing[season_csv_to_array[i][5]] += 3
    i=i+1


In [25]:
print(standing)

{'Sheffield United': 54, 'Wolverhampton Wanderers': 39, 'West Ham United': 51, 'Norwich City': 51, 'Tottenham Hotspur': 63, 'Southampton': 51, 'Aston Villa': 51, 'Liverpool': 75, 'Manchester United': 69, 'Crystal Palace': 54, 'Chelsea': 72, 'Brighton and Hove Albion': 57, 'Manchester City': 81, 'Leicester City': 81, 'AFC Bournemouth': 48, 'Newcastle United': 48, 'Watford': 45, 'Burnley': 39, 'Everton': 57, 'Arsenal': 54}


In [26]:
sorted_dict = dict(sorted(standing.items(), key=lambda x: x[1], reverse=True))

In [27]:
print(sorted_dict)

{'Manchester City': 81, 'Leicester City': 81, 'Liverpool': 75, 'Chelsea': 72, 'Manchester United': 69, 'Tottenham Hotspur': 63, 'Brighton and Hove Albion': 57, 'Everton': 57, 'Sheffield United': 54, 'Crystal Palace': 54, 'Arsenal': 54, 'West Ham United': 51, 'Norwich City': 51, 'Southampton': 51, 'Aston Villa': 51, 'AFC Bournemouth': 48, 'Newcastle United': 48, 'Watford': 45, 'Wolverhampton Wanderers': 39, 'Burnley': 39}


In [28]:
standing = {}
indexofteams = []
for i in range(len(season_csv_to_array)):
    season_number = (int)(season_csv_to_array[i][2][0] + season_csv_to_array[i][2][1])
    if season_number == 20:
        indexofteams.append(i)
        if (season_csv_to_array[i][4]) in standing:
            standing[season_csv_to_array[i][4]] = 0
        else:
            standing[season_csv_to_array[i][4]] = 0

In [29]:
i= 3799
for ind in range(len(y_pred2)):
    if y_pred2[ind] == 1:
        standing[season_csv_to_array[i][4]] += 3
    elif y_pred2[ind] == 0:
        standing[season_csv_to_array[i][4]] += 1
        standing[season_csv_to_array[i][5]] += 1
    else:
        standing[season_csv_to_array[i][5]] += 3
    i=i+1

In [46]:
sorted_dict = dict(sorted(standing.items(), key=lambda x: x[1], reverse=True))
print(sorted_dict)
ct = 0
for it in sorted_dict:
    print(ct, '\t', it, ':', sorted_dict[it])
    ct = ct+1

{'Manchester City': 66, 'Everton': 54, 'Leicester City': 51, 'Brighton and Hove Albion': 48, 'Wolverhampton Wanderers': 45, 'Chelsea': 45, 'Liverpool': 45, 'Manchester United': 42, 'Leeds United': 42, 'Southampton': 42, 'West Ham United': 42, 'Newcastle United': 42, 'Tottenham Hotspur': 39, 'Arsenal': 36, 'Fulham': 36, 'Crystal Palace': 33, 'Aston Villa': 30, 'West Bromwich Albion': 27, 'Sheffield United': 24, 'Burnley': 24}
0 	 Manchester City : 66
1 	 Everton : 54
2 	 Leicester City : 51
3 	 Brighton and Hove Albion : 48
4 	 Wolverhampton Wanderers : 45
5 	 Chelsea : 45
6 	 Liverpool : 45
7 	 Manchester United : 42
8 	 Leeds United : 42
9 	 Southampton : 42
10 	 West Ham United : 42
11 	 Newcastle United : 42
12 	 Tottenham Hotspur : 39
13 	 Arsenal : 36
14 	 Fulham : 36
15 	 Crystal Palace : 33
16 	 Aston Villa : 30
17 	 West Bromwich Albion : 27
18 	 Sheffield United : 24
19 	 Burnley : 24
